In [1]:
import mediapipe as mp # Import mediapipe
import cv2 # Import opencv
import csv
import os
import numpy as np
import pandas as pd
import time
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline 
from sklearn.preprocessing import StandardScaler 

from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score # Accuracy metrics 
import pickle 

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
mp_holistic = mp.solutions.holistic
mp_drawing_styles = mp.solutions.drawing_styles

In [3]:
def calculate_angle(a,b):
    a = np.array(a)
    b = np.array(b)
    
    radians = np.arctan2(b[1]-a[1], b[0]-a[0])
    angle = np.abs(radians*180.0/np.pi)

    return angle 

In [57]:
def are_last_two_elements_equal(arr):
    if not arr or len(arr) < 2:
        return False
    return arr[-2] == arr[-1]

In [58]:
are_last_two_elements_equal([1])

False

In [4]:
def time_checking(lst,hand):
    if any(lst):
        hand=True
        

In [33]:
cap = cv2.VideoCapture('../../Downloads/mhchoi.mp4')

anomaly = {'shoulder':[],'left_hand':False,'right_hand':False}
left_coords={'left_wrist':[],'left_pinky':[],'left_index':[],'left_thumb':[]}
right_coords={'right_wrist':[],'right_pinky':[],'right_index':[],'right_thumb':[]}

# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break 
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        # print(results.face_landmarks)
        
        # face_landmarks, pose_landmarks, left_hand_landmarks, right_hand_landmarks
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # # 1. Draw face landmarks
        # mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS, 
        #                         mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1),
        #                         mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
        #                         )
        
        # 2. Right hand
        mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                                mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                                )

        # 3. Left Hand
        mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                                mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                                mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                                )

        # 4. Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                )
        # Export coordinates
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get shoulder coordinates
            left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].y]
            right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].y]
            
            # Calculate shoulder angle
            angle = calculate_angle(left_shoulder, right_shoulder)
            if angle<=170: # 기준 정해야함
                anomaly['shoulder'].append(time.time())
            else:
                shoulder_seconds = anomaly["shoulder"][-1]-anomaly["shoulder"][0]
                if shoulder_seconds >= 1e-3:
                    print(f'{shoulder_seconds:.3f}초 동안 자세가 좋지 않았습니다.')
                anomaly['shoulder']=[]

            
            # Get Hand Components' Coordinates
            landmarks_list = [mp_pose.PoseLandmark.LEFT_WRIST, mp_pose.PoseLandmark.LEFT_PINKY, mp_pose.PoseLandmark.LEFT_INDEX, mp_pose.PoseLandmark.LEFT_THUMB, mp_pose.PoseLandmark.RIGHT_WRIST, mp_pose.PoseLandmark.RIGHT_PINKY, mp_pose.PoseLandmark.RIGHT_INDEX, mp_pose.PoseLandmark.RIGHT_THUMB]

for landmark in landmarks_list:
    variable_name = landmark.lower().replace("_", "")
    variable_name = landmarks[landmark]



        except:
            pass
                        
        cv2.imshow('Video Feed', image)
        if cv2.waitKey(10) & 0xFF == ord("q"):
            break

cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)
cv2.waitKey(1)
cv2.waitKey(1)
cv2.waitKey(1)

0.060초 동안 자세가 좋지 않았습니다.
0.223초 동안 자세가 좋지 않았습니다.


-1

In [46]:
landmarks[mp_pose.PoseLandmark.LEFT_WRIST]

x: 0.9589765667915344
y: 1.5836498737335205
z: -0.9355209469795227
visibility: 0.08272692561149597

In [39]:
any([1,2,3]) == 4

False